In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(
    api_key = OPENAI_API_KEY
)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role':'user', 'content':'hi'}],
    temperature=0
)

print(completion.choices[0].message.content)


Hello! How can I assist you today?


## 데이터 전처리 함수 작성
리뷰 JSON 파일 로드

In [11]:
import json

with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)
    
review_list[:3]

[{'review': '서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍',
  'stars': 5,
  'date': '2024.01.24'},
 {'review': '한강뷰가 잘 안보인다는 후기가 있어 걱정했는데 통창으로 너무 잘보였어요!!!! 짱짱!!!',
  'stars': 5,
  'date': '1일 전'},
 {'review': '서울 출장으로 편하게 쉬었습니다', 'stars': 5, 'date': '2일 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별점 5개
- 나쁜 평점 = 별점 4개 이하

In [12]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

# 좋은 리뷰, 나쁜 리뷰
good_cnt, bad_cnt

(188, 32)

좋은 리뷰 및 나쁜 리뷰로 구분후 저장

In [13]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]깨끗하고 좋앗어요\n다음에 또가고 싶어요[REVIEW_END]',
 '[REVIEW_START]좋아요~~다음에 재방문할게요[REVIEW_END]',
 '[REVIEW_START]이런 집에서 살고 싶은 청와대 주변과 매우 잘 어울리는 숙소\n\n입구에서 나는 향이 좋고 방 디자인이 미국식 인테리어일거 같다\n미군들도 많이 이용하는거 같다\n\n훌륭하다[REVIEW_END]']

In [14]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]깨끗하고 좋앗어요\n다음에 또가고 싶어요[REVIEW_END]\n[REVIEW_START]좋아요~~다음에 재방문할게요[REVIEW_END]\n[REVIEW_ST'

전처리 함수 작성

In [16]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
        
    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    
    for r in review_list:
        review_date_str = r['date']
        # 몇 일전, 몇 시간전 등의 예외 케이스를 대응하기 위한 try~excpet
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            # 최대 일주일간은 현재 날짜로 대체
            review_date = current_date
        
        if review_date < date_boundary:
            continue
        
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
            
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    
    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]한강뷰가 잘 안보인다는 후기가 있어 걱정했는데 통창으로 너무 잘보였어요!!!! 짱짱!!![REVIEW_END]\n[REVIEW_START]서울 출장으로 편하'

## 평가용 함수 작성
평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
    - 비교하는 방식 VS 점수 매기는 방식
    - 점수라는것이 애매할 수 있음(EX. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정

In [18]:
def pairwise_eval(reviews, answer_a, answer_b):
    # 원래는 뉴스 비교 프롬프트 지만 조금 수정함
    # 1. x -> korean summaries
    # 2. news -> accommodations
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""

    completion = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role':'user', 'content': eval_prompt}],
        temperature=0
    )
    
    return completion

## Baseline 모델 개발

In [19]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [ ]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo'):
    prompt = prompt + '\n\n' + reviews
    
    completion = client.chat.completions.create(
         model=model,
         messages=[{'role':'user', 'content': prompt}],
         temperature=temperature
    )
    
    return completion